In [ ]:
import sys
%load_ext autoreload
%autoreload 2

#### summed_like_tool can be obtained from github: https://github.com/mireianievas/summed_like_tool
#### shame on me for the complete lack of documentation. 

sys.path.append("/home/mnievas/Software/summed_like_tool/")
#from summed_like_tool import fermi,lst,utils,fitmaker
import astropy.units as u
import regions
import matplotlib.pyplot as plt
from summed_like_tool.fermi import analysis as fermianalysis
from summed_like_tool.fitmaker import spectrum as spectrum
from gammapy.maps import WcsGeom
from gammapy.modeling.models import (
    SkyModel,
    EBLAbsorptionNormSpectralModel,
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
)

In [ ]:
%env GAMMAPY_DATA=/home/mnievas/Software/cta/gammapy/gammapy-datasets/

In [ ]:
redshift = 0.00428

dominguez = EBLAbsorptionNormSpectralModel.read_builtin("dominguez", redshift=redshift)
lat_path = "/home/mnievas/LAT/M87_2020-2021/"
lat_aux  = "/home/mnievas/LAT/enrico/Data/diffuse/"
srcmodel = 'PowerLaw'
targetname = "M87"

In [ ]:
FermiFrontBack = fermianalysis.FermiAnalysis(lat_path,lat_aux,srcmodel)
FermiFrontBack.prepare_files('FRONT')

In [ ]:
FermiFrontBack.print_selected_files()

In [ ]:
FermiFrontBack.gen_analysis(targetname,redshift=redshift,ebl_model='dominguez')

In [ ]:
FermiFrontBack.plot_counts_map()
for s in FermiFrontBack.SkyModel.list_sources:
    spatial = s.spatial_model
    if (spatial.tag[0]=='PointSpatialModel'):
        for c in spatial.parameters:
            if c.name == 'lon_0':
                lon = c.value
            if c.name == 'lat_0':
                lat = c.value
        #print(lon,lat,c)
        plt.gca().plot_coord(spatial.position,color='red',marker='x',ms=3)
        #break


In [ ]:
# Filter out data far away from the source position in the minimization
exclusion_region = regions.CircleAnnulusSkyRegion(FermiFrontBack.dataset.mask_image.geom.center_skydir,
                                          10*u.deg,30*u.deg)

FermiFrontBack.add_exclusion_region(exclusion_region)
FermiFrontBack.create_exclusion_mask()

In [ ]:
FermiFrontBack.exclusion_mask.plot_interactive()

In [ ]:
FSpectrum = spectrum.SpectralAnalysis([FermiFrontBack])
FSpectrum.set_energy_mask(FSpectrum.datasets[0],emin=100*u.MeV,emax=1000*u.GeV)
FSpectrum.global_fit()
FSpectrum.set_target_source(targetname)
FSpectrum.read_enrico_spectrum()
FSpectrum.get_spectral_points()
FSpectrum.plot_spectrum()
FSpectrum.plot_spectrum_enrico()

In [ ]:
srcpos = FermiFrontBack.SkyModel.list_sources[-1].spatial_model.position
FermiFrontBack.dataset.plot_residuals(kwargs_spectral={'region':regions.CircleSkyRegion(srcpos,radius=3*u.deg)})

### MAGIC 

In [ ]:
from summed_like_tool.iact import analysis as iactanalysis
import os
import glob
import numpy as np
from gammapy.data import DataStore
from pathlib import Path
import astropy.io.fits as pyfits

In [ ]:
#!rm -rf /home/mnievas/DL3/M87/MAGIC/all/20*DL3*.fits
#!rm -rf /home/mnievas/DL3/M87/MAGIC/fold/20*DL3*.fits

In [ ]:
#!ls /mnt/sda1/MAGIC/M87/M87_ST0316/M87/DL3/*/20*DL3*.fits | wc -l

In [ ]:
#!cp -a /mnt/sda1/MAGIC/M87/M87_ST0316/M87/DL3/*/20*DL3*.fits /home/mnievas/DL3/M87/MAGIC/all/
#!cp -a /home/mnievas/DL3/M87/MAGIC/{Dark,Light,Moderate,Bright,VeryBright}/*DL3*.fits /home/mnievas/DL3/M87/MAGIC/all/

In [ ]:
#!ls /home/mnievas/DL3/M87/MAGIC/all/2*.fits | wc -l

In [ ]:
#!rsync -av -L /mnt/sda1/MAGIC/M87/M87_ST0316/M87/Flute/*{fold,foam,combine,Fold,Foam,Combine}* /home/mnievas/DL3/M87/MAGIC/fold/
#!micsync mic:~/CurrentSource/M87/Flute/*{fold,foam,combine,Fold,Foam,Combine}* /home/mnievas/DL3/M87/MAGIC/fold/

In [ ]:
magicpath = "/home/mnievas/DL3/M87/MAGIC/all/"
paths = glob.glob(f"{magicpath}/2*DL3*.fits")

In [ ]:
len(paths)

In [ ]:
pyfits.open(paths[0])[3].header

In [ ]:
magicpath = "/home/mnievas/DL3/M87/MAGIC/all/"
#paths = glob.glob(f"{magicpath}/20*DL3*.fits")
#data_store = DataStore.from_events_files(paths)
#observations = data_store.get_observations(required_irf=["psf","aeff", "edisp","rad_max"])
#data_store.hdu_table.write("hdu-index.fits.gz",overwrite=True) 
#data_store.obs_table.write("obs-index.fits.gz",overwrite=True) 

In [ ]:
srcmodel = "PowerLaw"
MAGIC = iactanalysis.Analysis1D(magicpath,srcmodel)
MAGIC.create_datastore(overwrite=True)
#MAGIC.get_targetname()
MAGIC.set_targetname("M87")
MAGIC.get_observations()
MAGIC.set_energy_axes()
MAGIC.set_src_pos()
MAGIC.set_on_region()
MAGIC.run_region_finder(1)
MAGIC.create_safe_mask_min_aeff(
    aeff_percent=5,
    bias_percent=5
)
MAGIC.add_exclusion_region()
MAGIC.create_exclusion_mask()
MAGIC.create_datasets()
spectral_model_lp = PowerLawSpectralModel(
    amplitude=1e-10 * u.Unit("cm-2 s-1 TeV-1"),
    index=2.2,
    #beta=0.02,
    reference=0.5*u.TeV,
)
magic_model = SkyModel(spectral_model=spectral_model_lp, 
                     name=MAGIC.targetname)

from gammapy.datasets import (
    Datasets,
)

MAGIC.dataset = Datasets(MAGIC.datasets).stack_reduce(name='MAGIC')

MAGIC.dataset.models = [magic_model]
#MAGIC.dataset.models = [FermiFrontBack.dataset.models[-1]]


In [ ]:
MAGIC.plot_excess_ts_livetime()

In [ ]:
myobs = MAGIC.observations_total[0]

In [ ]:
from gammapy.makers.utils import make_theta_squared_table
from gammapy.visualization import plot_theta_squared_table
from gammapy.maps import MapAxis

theta2cut_f = 0.009
thetacut  = np.sqrt(theta2cut_f)

theta2_axis = MapAxis.from_bounds(0, theta2cut_f*20, nbin=40, interp="lin", unit="deg2")

theta2_table = make_theta_squared_table(
    observations=MAGIC.observations_total,
    position=MAGIC.src_pos,
    theta_squared_axis=theta2_axis,
)
#plt.figure(dpi=120)
#plot_theta_squared_table(theta2_table)

plt.figure(figsize=(6,3),dpi=120)
plt.stairs(
    edges=np.append(theta2_table['theta2_min'],
                    theta2_table['theta2_max'][-1]),
    values=theta2_table['counts_off'],
    fill=True,
    color='0.2',
    facecolor='0.8',
    label='OFF data'
)

plt.stairs(
    edges=np.append(theta2_table['theta2_min'],
                    theta2_table['theta2_max'][-1]),
    values=theta2_table['counts_off'],
    fill=False,
    color='0.2',
    lw=0.5,
)

plt.errorbar(
    x = 0.5*(theta2_table['theta2_min']+theta2_table['theta2_max']),
    xerr = 0.5*(theta2_table['theta2_max']-theta2_table['theta2_min']),
    y = theta2_table['counts_off'] + theta2_table['excess'],
    yerr = [-theta2_table['excess_errn'],theta2_table['excess_errp'],],
    ls='None',
    color='darkblue',
    label='ON data',
    lw=0.5,
    marker='o',
    ms=1,
)


theta2cut = theta2_table['theta2_max']<=theta2cut_f

min_off_counts = np.min(theta2_table['counts_off'])
min_on_counts = np.min(theta2_table['counts']+theta2_table['excess_errn'])


TStot = np.sum(theta2_table['sqrt_ts'][theta2cut]*\
               np.abs(theta2_table['sqrt_ts'][theta2cut]))

plt.axvline(theta2_table['theta2_max'][theta2cut][-1],
            ls='dotted',
            ymin=0.2,ymax=0.8,
            color='0.2',label='$\\theta^2 cut$',lw=1)

plt.ylim(np.min([min_off_counts*0.95,min_on_counts*0.98]),)
plt.legend()
plt.xlabel('$\\theta^2$ [deg^2]')
plt.xlim(0,theta2cut_f*20)
plt.ylabel('$\mathrm{N_{events}}$')
plt.text(0.4,0.9,f'Significance = {np.sqrt(TStot):.1f} $\sigma$',transform=plt.gca().transAxes )

In [ ]:
!ls /home/mnievas/DL3/M87/MAGIC/fold/

In [ ]:
import uproot
foam = uproot.open("/home/mnievas/DL3/M87/MAGIC/fold/foam.root")

In [ ]:
sss = foam['total_non']

In [ ]:
sss.to_numpy()

In [ ]:
scipy_opts = {
    "method": "L-BFGS-B",
    "options": {"ftol": 1e-4, "gtol": 1e-05},
    "backend": "scipy",
}

MSpectrum = spectrum.SpectralAnalysis([MAGIC],backend='scipy',optimize_opts = scipy_opts,store_trace=True)
mod = MSpectrum.datasets[0].models
mod.parameters.value = [2.3, 2e-12, 0.5]
mod.parameters.max   = [3.5, 5e-11, 0.5]
mod.parameters.min   = [1.5, 1e-13, 0.5]
mod.parameters[0].error = 0.2
mod.parameters[1].error = 1e-12
print(mod.parameters.to_table())
#MSpectrum.global_fit()
MSpectrum.set_target_source(targetname)
MSpectrum.set_energy_mask(MSpectrum.datasets[0],emin=80*u.GeV,emax=10*u.TeV)
MSpectrum.global_fit()

MSpectrum.target_model = MAGIC.dataset.models[0]
MSpectrum.get_spectral_points(np.logspace(np.log10(50),np.log10(20000),17)*u.GeV)
MSpectrum.plot_spectrum()
MSpectrum.plot_spectrum_fold(f"{magicpath.replace('all','fold')}/Output_fold.root")

In [ ]:
MSpectrum.fit.minuit

In [ ]:
print(MAGIC.dataset.models[-1])

In [ ]:
from gammapy.estimators import LightCurveEstimator
import astropy.time as atime

lc_maker = LightCurveEstimator(
    energy_edges=[0.35, 20] * u.TeV, source="M87-MAGIC", reoptimize=False,
)


magic_lc = lc_maker.run(MAGIC.dataset)
magic_lc_table = magic_lc.to_table(sed_type="flux", 
                                   format="lightcurve")

print(magic_lc_table)
#magic_lc.plot(color="tab:orange")
#ax = magic_lc.plot_ts_profiles()


In [ ]:
FMSpectrum = spectrum.SpectralAnalysis([FermiFrontBack,MAGIC])
#FMSpectrum.global_fit()
#FMSpectrum.set_target_source(targetname)
#FMSpectrum.set_energy_mask(FMSpectrum.datasets[0],emin=80*u.GeV,emax=10*u.TeV)
#FMSpectrum.target_model = MAGIC.dataset.models[0]
#FMSpectrum.get_spectral_points(np.logspace(1.8,4,13)*u.GeV)
#FMSpectrum.plot_spectrum()
#FMSpectrum.plot_spectrum_fold(f"{magicpath.replace('all','fold')}/Output_fold.root")

In [ ]:
spectral_model_lp = LogParabolaSpectralModel(
    amplitude=1e-9 * u.Unit("cm-2 s-1 TeV-1"),
    index=1.9,
    beta=0.02,
    reference=10*u.GeV,
)

In [ ]:
from gammapy.modeling.models.core import DatasetModels
import astropy.table as tab

models = DatasetModels(FermiFrontBack.dataset.models)
models[-1].spectral_model = spectral_model_lp

for model in models:
    model.datasets_names = ['Fermi-LAT_FRONT']
    
models[-1].datasets_names = ['Fermi-LAT_FRONT','MAGIC']


#model_1d = SkyModel(
#    spectral_model=models[-1].spectral_model,
#    name="M87-MAGIC",
#    datasets_names=["MAGIC"],
#)
#models = models + model_1d


In [ ]:
FMSpectrum.datasets.models = models

In [ ]:
print(FMSpectrum.datasets)

In [ ]:
print(models[-1])

In [ ]:
FMSpectrum.set_energy_mask(FMSpectrum.datasets[0],emin=100*u.MeV,emax=30*u.TeV)
FMSpectrum.set_energy_mask(FMSpectrum.datasets[1],emin=100*u.MeV,emax=30*u.TeV)
FMSpectrum.global_fit()

In [ ]:
FMSpectrum.ebin_edges = [0.1*u.GeV,30*u.TeV]
FMSpectrum.set_target_source(FMSpectrum.datasets.models[-1].name)

In [ ]:
FMSpectrum.plot_spectrum()

In [ ]:
FMSpectrum.datasets["MAGIC"].npred().data.flatten()

In [ ]:
FMSpectrum2 = spectrum.SpectralAnalysis([FermiFrontBack,MAGIC])
from gammapy.modeling.models.core import DatasetModels
from gammapy.modeling.models import EBLAbsorptionNormSpectralModel
import astropy.table as tab

models2 = DatasetModels(FermiFrontBack.dataset.models)

redshift = 0.00428
eblabs = EBLAbsorptionNormSpectralModel.read_builtin("dominguez")

# EBL + PWL model
spectral_model_lp_abs = spectral_model_lp * eblabs

models2[-1].spectral_model = spectral_model_lp_abs

for model in models2:
    model.datasets_names = ['Fermi-LAT_FRONT']
    
#models2[-1].datasets_names = ['Fermi-LAT_FRONT','MAGIC']


model_1d = SkyModel(
    spectral_model=models2[-1].spectral_model,
    name="M87-MAGIC",
    datasets_names=["MAGIC"],
)
models2 = models2 + model_1d

FMSpectrum2.datasets.models = models2

FMSpectrum2.set_energy_mask(FMSpectrum2.datasets[0],emin=100*u.MeV,emax=30*u.TeV)
FMSpectrum2.set_energy_mask(FMSpectrum2.datasets[1],emin=100*u.MeV,emax=30*u.TeV)
FMSpectrum2.global_fit()

FMSpectrum2.ebin_edges = [0.1*u.GeV,30*u.TeV]
FMSpectrum2.set_target_source(FMSpectrum2.datasets.models[-2].name)
FMSpectrum2.plot_spectrum()
FMSpectrum2.ax.set_xlim([0.08,30000])
FMSpectrum2.ax.set_ylim([1e-15,4e-11])
#FMSpectrum2.Fig

In [ ]:
FMSpectrum2.datasets["MAGIC"].npred().data.flatten()

In [ ]:
FMSpectrum2.set_target_source(FMSpectrum.datasets.models[-1].name,FMSpectrum2.datasets[-1])
FMSpectrum2.get_spectral_points(np.logspace(2,4,8)*u.GeV,datasets=[FMSpectrum2.datasets[-1]])
fptab1 = FMSpectrum2.flux_points

In [ ]:
FMSpectrum2.set_target_source(FMSpectrum2.datasets.models[-2].name,FMSpectrum2.datasets[-2])
FMSpectrum2.get_spectral_points(np.logspace(-1,3,9)*u.GeV,datasets=[FMSpectrum2.datasets[-2]])
fptab2 = FMSpectrum2.flux_points

In [ ]:
fptab1.plot(ax=FMSpectrum2.ax,sed_type='e2dnde')
fptab2.plot(ax=FMSpectrum2.ax,sed_type='e2dnde')

In [ ]:
FMSpectrum2.ebin_edges = [0.1*u.GeV,30*u.TeV]
FMSpectrum2.set_target_source(FMSpectrum2.datasets.models[-2].name)
FMSpectrum2.ax.set_xlim([0.08,30000])
FMSpectrum2.ax.set_ylim([1e-15,4e-11])
FMSpectrum2.Fig

In [ ]:
FMSpectrum2.plot_spectrum_fold(f"{magicpath.replace('all','fold')}/Output_fold.root")

In [ ]:
FMSpectrum2.lat_bute = FSpectrum.lat_bute
FMSpectrum2.lat_ebin = FSpectrum.lat_ebin
FMSpectrum2.plot_spectrum_enrico()

In [ ]:
FMSpectrum2.ax.set_xlim([0.08,30000])
FMSpectrum2.ax.set_ylim([3e-14,4e-11])

In [ ]:
FMSpectrum2.Fig

In [ ]:
mod = FMSpectrum2.datasets.models

In [ ]:
#for f in mod:
#    if 'M87' not in f.name:
#        f.freeze()

In [ ]:
#for f in mod.parameters.free_parameters:
#    print(f)

In [ ]:
print(mod[-1])

In [ ]:
FMSpectrum2.datasets.models[-2].datasets_names = ['Fermi-LAT_FRONT','MAGIC']

In [ ]:
FMSpectrum2.set_target_source(FMSpectrum2.datasets.models[-2].name,FMSpectrum2.datasets[0])
FMSpectrum2.get_spectral_points(np.logspace(-1,4.4,18)*u.GeV)
fptab3 = FMSpectrum2.flux_points

In [ ]:
fptab3.plot(ax=FMSpectrum2.ax,sed_type='e2dnde')
FMSpectrum2.ax.set_xlim([0.08,30000])
FMSpectrum2.ax.set_ylim([3e-14,4e-11])
FMSpectrum2.Fig

In [ ]:
FMSpectrum2.datasets.models